# Utilisation des Handoffs

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

llm_model = "gpt-4o-mini"

In [ ]:
from agents import Agent, handoff

billing_agent = Agent(name="Billing agent", model=llm_model)
refund_agent = Agent(name="Refund agent", model=llm_model)


triage_agent = Agent(name="Triage agent", model=llm_model,handoffs=[billing_agent, handoff(refund_agent)])

In [ ]:
from agents import Runner

result = await Runner.run(triage_agent, "I want a refund for my last purchase")

In [ ]:
print(result)

In [ ]:
print(result.new_items[0])

Pour visualiser le graph d'appel, il est nécessaire d'installer GraphViz

In [ ]:
from agents.extensions.visualization import draw_graph
draw_graph(triage_agent)

In [ ]:
from pydantic import BaseModel

from agents import Agent, handoff, RunContextWrapper

class EscalationData(BaseModel):
    reason: str

async def on_handoff(ctx: RunContextWrapper[None], input_data: EscalationData):
    print(f"Escalation agent called with reason: {input_data.reason}")

agent = Agent(name="Escalation agent")

handoff_obj = handoff(
    agent=agent,
    on_handoff=on_handoff,
    input_type=EscalationData,
)

In [ ]:
triage_agent.handoffs.append(handoff_obj)

In [ ]:
draw_graph(triage_agent)

In [ ]:
result = await Runner.run(triage_agent, "I want to escalate my issue because I am not satisfied with the refund process")

In [ ]:
print(result.last_agent)

In [ ]:
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions 

handoff_obj.agent(
    name = "Escalation agent",
    instructions=prompt_with_handoff_instructions("Answer politely and provide a solution to the customer's issue.")
)